In [1]:
from typing import Optional
from datetime import datetime
from pydantic import BaseModel, Field
from openai import OpenAI
import requests
import logging
import json
from collections import defaultdict
from datetime import datetime

In [2]:
client = client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

model = "llama3.2:3b"

In [3]:
class check_prompt(BaseModel):
    """First LLM call: Extract basic accomodation information"""

    # description: str = Field(description="Raw description of the event")
    is_accomodation_search: bool = Field(
        description="Is the text a query related to finding an accomodation?"
    )


class EventDetails(BaseModel):
    """Second LLM call: Parse specific event details"""

    city: str = Field(description="City in which the accomodation should be booked")
    start_date: str = Field(
        description="Date and time of checking into the accomodation. Strictly use 'yyyy-mm-dd' date format for this value. Example is 2025-03-25"
    )
    end_date: str = Field(
        description="Date and time of checking out of the accomodation. Strictly use 'yyyy-mm-dd' date format for this value. Example is 2025-03-25"
    )

In [4]:
def check_accomodation_request(input: str):
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are an assistant for finding accomodation. \
                    You are allowed to answer only to queries related to accomodation search. \
                    In case of other requests, you must decline. \
                    In this context, Analyze if the text describes a query for finding accomodation. \
                    Say yes if the text describes a query for finding accomodation, else say no",
            },
            {"role": "user", "content": input},
        ],
        response_format=check_prompt
    )

    return completion.choices[0].message.parsed.is_accomodation_search


def extract_stay_details(input:str):
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"You are an assistant for finding accomodation. \
                    Extract the name of the city in which the user wishes to stay. If city is not found, strictly use the placeholder [None].\
                    Extract the check-in date. Check-in date is date on which the user will start staying in the accomodation. Strictly use 'yyyy-mm-dd' date format. If check-in date is not found, strictly use the placeholder [None].\
                    Extract the check-out date. Check-out date is date on which the user will leave the accomodation.  Strictly use 'yyyy-mm-dd' date format. If check-out date is not found, strictly use the placeholder [None]. \
                    Do not make any assumptions for the city, check-in date and check-out date.",
            },
            {"role": "user", "content": input},
        ],
        response_format=EventDetails
    )

    return completion

In [5]:
def process_accomodation_search(input: str):
    is_accomodation_request = check_accomodation_request(input)

    if not is_accomodation_request:
        return None
    else:
        stay_details = extract_stay_details(input)
        return stay_details

In [6]:
user_input = "Suggest me some hotels in Bengaluru for staying from 27th March 2025 and 31st March 2025"
# user_input = "Suggest me some hotels in Bengaluru"
# user_input = "Write a poem"

In [7]:
result = process_accomodation_search(user_input)

if not result:
    print("This is not an accomodation search request!")
else:
    pass

In [8]:
place_of_stay = result.choices[0].message.parsed.city
check_in_date = result.choices[0].message.parsed.start_date
check_out_date = result.choices[0].message.parsed.end_date

In [9]:
place_of_stay, check_in_date, check_out_date

('Bengaluru', '2025-03-27', '2025-03-31')

In [10]:
def get_weather_forecast(latitude, longitude, start_date, end_date):
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,relative_humidity_2m,rain&start_date={start_date}&end_date={end_date}"
    )
    data = response.json()
    
    time = data['hourly'].get('time', [])
    temperature = data['hourly'].get('temperature_2m', [])
    humidity = data['hourly'].get('relative_humidity_2m', [])
    rain = data['hourly'].get('rain', [])

    if not (time and temperature and humidity and rain):
        return {"error": "Incomplete weather data."}

    daily_data = defaultdict(lambda: {'temperature': [], 'humidity': [], 'total_rain': 0, 'rain_times': []})

    for i in range(len(time)):
        date = datetime.fromisoformat(time[i]).date()
        daily_data[date]['temperature'].append(temperature[i])
        daily_data[date]['humidity'].append(humidity[i])
        daily_data[date]['total_rain'] += rain[i]

        if rain[i] > 0:
            daily_data[date]['rain_times'].append({"time": time[i].split('T')[1], "rain": rain[i]})

    result = {}
    for date, values in daily_data.items():
        avg_temp = sum(values['temperature']) / len(values['temperature'])
        avg_humidity = sum(values['humidity']) / len(values['humidity'])
        result[str(date)] = {
            "average_temperature": round(avg_temp, 1),
            "average_humidity": round(avg_humidity, 1),
            "total_rain": round(values['total_rain'], 1),
            "rain_times": values['rain_times']
        }

    return result, data

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather_forecast",
            "description": "Get the forecast for average temperature in Celsius, average relative humidity in percentage, \
                total rain in mm and timing of rain for provided coordinates between the start date and end date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                    "start_date": {"type": "string"},
                    "end_date": {"type": "string"}
                },
                "required": ["latitude", "longitude", "start_date", "end_date"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [12]:
system_prompt = "You are a helpful weather assistant. \
                You can determine the latitude and longitude based on the name of the city/place \
                You strictly use 'yyyy-mm-dd' format for dates. \
                Based on the temperature, relative humidity and rain data, you can provide a concise report on the weather in natural language."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"What is the weather forecast for {place_of_stay} between {check_in_date} and {check_out_date}?"},
]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

In [13]:
completion.model_dump()

{'id': 'chatcmpl-712',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': '',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_rfyeklmg',
      'function': {'arguments': '{"end_date":"2025-03-31","latitude":"12.9848","longitude":"77.5856","start_date":"2025-03-27"}',
       'name': 'get_weather_forecast'},
      'type': 'function',
      'index': 0}]}}],
 'created': 1742915957,
 'model': 'llama3.2:3b',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_ollama',
 'usage': {'completion_tokens': 55,
  'prompt_tokens': 304,
  'total_tokens': 359,
  'completion_tokens_details': None,
  'prompt_tokens_details': None}}

In [14]:
def call_function(name, args):
    if name == "get_weather_forecast":
        return get_weather_forecast(**args)

In [15]:

for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result, _ = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [16]:

class WeatherResponse(BaseModel):
    weather_report: str = Field(
        description="A text summary on the weather in natural language based on the weather forcast data")

In [17]:
completion_2 = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

In [18]:
print(completion_2.choices[0].message.parsed.weather_report)

Bengaluru's weather forecast between 2025-03-27 and 2025-03-31 shows a general trend of increasing temperature with decreasing relative humidity as the period progresses. On these days, there is no significant rainfall expected, making it suitable for outdoor activities. As most of these days have average temperatures between 25.6°C to 28.0°C and low humidity levels, wearing light clothing would be a good idea. It's also worth noting that humidity will vary a little each day but is never quite too high as to make one feel uncomfortable outdoors.
